<a href="https://colab.research.google.com/github/Aleyucra74/Colab-google-MLP-OCR/blob/main/tratamento_de_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ideia número 1:
Reconhecimento por palavras do município

Passos: Leitura do pdf pelo OpenCV, tirando foto do cabeçalho 

Leitura do pytesseract das palavras dessa foto, salvando em tuplas esse conteúdo já com o nome do município para treino

Organizar essa base e colocar no lugar dos arquivos train, validation, test conforme já tem exemplo.

Utilizar modelo ML Perceptron no exemplo  

Avaliar acurácia (A meta é 50~75%)

Colocar modelo em operação com função joblib

Ideia número 2:

Não usar machine learning, e sim a função thefuzz que agrupa palavras parecidas.













In [ ]:
#instalando tesseract
!sudo apt install tesseract-ocr
!pip install pytesseract pdf2image thefuzz poppler-utils PyMuPDF==1.18.9

In [ ]:
# usando opencv , tesseract , thefuzz
import pytesseract
from PIL import Image
import cv2
import re
from thefuzz import fuzz
from thefuzz import process
import fitz

import os

In [ ]:
#############################################
# SOMENTE DEPOIS (gpu para acelerar processamento)
#############################################
# from h2o4gpu.model_selection import train_test_split
# from h2o4gpu.neural_network import MLPClassifier
# from h2o4gpu.preprocessing import StandardScaler
# from h2o4gpu.metrics import mean_squared_error, confusion_matrix, accuracy_score, precision_score, recall_score
#############################################

#Importando a técnica Multi Layer Perceptron do Sklearn
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score, precision_score, recall_score
from matplotlib import pyplot as plt
from collections import defaultdict
from datetime import datetime
import multiprocessing
import pandas as pd
import numpy as np
import joblib
import cudf 
import cupy

In [ ]:
#lendo palavras do pdf>imagem>palavras (opção 1)
def proc():
    main_text = ''
    for imgn in os.listdir('imgs'):
        img = cv2.imread(f'imgs/{imgn}')
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        file = f'grays\{imgn}.png'
        cv2.imwrite(file,gray)
        text=pytesseract.image_to_string(Image.open(file))
        main_text += text.replace('\n', '') + '\n'
    return main_text

In [ ]:
#lendo palavras do pdf>imagem>palavras (opção 2) (testar)
def proc():
    main_text = ''
    for imgn in os.listdir('imgs'):
        img = cv2.imread(f'imgs/{imgn}')
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        text=pytesseract.image_to_string(gray)
        main_text += text.replace('\n', '') + '\n'
    return main_text

In [ ]:
#demonstração de como cortar pedaço do pdf (cabeçalho)
# from google.colab.patches import cv2_imshow # para visualizar imagem
def proc(imgn):
    img = cv2.imread(f'{imgn}')
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    crop_img1 = img[25:73, 250:843].copy()
    crop_img2 = img[10:170,20:230].copy() 
    # cv2_imshow(crop_img1) # para visualizar o corte 
    # cv2_imshow(crop_img2)
    text=pytesseract.image_to_string(crop_img1).replace('\n', '')
    return [text, crop_img2] # retorna texto encontrado e imagem do brazão

In [ ]:
#demonstração de tratamento do pdf para o tesseract ler melhor
def convert_pdf2img(input_file: str, pages:tuple = None):
    if not os.path.isdir('pasta_pdf'): os.mkdir('pasta_pdf')
    """Converts pdf to image and generates a file by page"""
    # Open the document
    pdfIn = fitz.open(input_file)
    output_files = []
    # Iterate throughout the pages
    for pg in range(pdfIn.pageCount):
        if pages is not None:
            if str(pg) not in str(pages):
                continue
        # Select a page
        page = pdfIn[pg]
        rotate = int(0)
        # PDF Page is converted into a whole picture 1056*816 and then for each picture a screenshot is taken.
        # zoom = 1.33333333 -----> Image size = 1056*816
        # zoom = 2 ---> 2 * Default Resolution (text is clear, image text is hard to read)    = filesize small / Image size = 1584*1224
        # zoom = 4 ---> 4 * Default Resolution (text is clear, image text is barely readable) = filesize large
        # zoom = 8 ---> 8 * Default Resolution (text is clear, image text is readable) = filesize large
        zoom_x = 2
        zoom_y = 2
        # The zoom factor is equal to 2 in order to make text clear
        # Pre-rotate is to rotate if needed.
        mat = fitz.Matrix(zoom_x, zoom_y).preRotate(rotate)
        pix = page.getPixmap(matrix=mat, alpha=False)
        output_file = f"pasta_pdf/{os.path.splitext(os.path.basename(input_file))[0]}_page{pg+1}.png"
        
        pix.writePNG(output_file)
        
        output_files.append(output_file)
    pdfIn.close()
    summary = {
        "File": input_file, "Pages": str(pages), "Output File(s)": str(output_files)
    }
    # Printing Summary
    print("## Summary ########################################################")
    print("\n".join("{}:{}".format(i, j) for i, j in summary.items()))
    print("###################################################################")
    return output_files

In [ ]:
# demonstração importando pasta inteira de notas 
textos = {}
for pdf in listdir('pasta_pdf'):
  img_pdf = convert_pdf2img(pdf)
  for imagem in img_pdf:
    textos[imagem] = proc(imagem)

In [ ]:
#demonstração de como salvar as palavras e as classes (municipios)

# palvras palaavras sao paulo palavras | classe1
# palvras palaavras sao paulo palavras | classe1
# palvras palaavras sao paulo palavras | classe1
# Treino 70% ~ 85% das notas 
# Teste 30% ~ 15% das notas
# (76% train - 12% validation - 12% test)

esperado = array_de_classes
treino = array_de_textos

teste = array_de_textos_teste
validacao = array_de_classes_teste

In [ ]:
#Modelo Multi Layer Perceptron

mlp = MLPClassifier(solver='lbfgs', alpha=0.001, max_iter=10000, hidden_layer_sizes=(1,2,3), learning_rate='adaptive')
mlp.fit(treino, esperado) # treina

output = mlp.predict(teste) # testa e classifica

mmlp['linhas_prc'].append(linhas)
mmlp['iteracao'].append((1,2,3)))
mmlp['precisao'].append(precision_score(validacao, output ))
mmlp['acuracia'].append(accuracy_score(validacao, output, normalize=False))
mmlp['acuracia_percent'].append(accuracy_score(validacao, output, normalize=False)/len(validacao))
mmlp['mse'].append(mean_squared_error(validacao, output))
mmlp['iters_to_fit'].append(mlp.n_iter_)
mmlp['matriz_confusao'].append(confusion_matrix(validacao, output))
mmlp['recal'].append(recall_score(validacao, output))
print('acuracia:', round(accuracy_score(validacao, output, normalize=False)/len(validacao), 4), '| precisao:', round(precision_score(validacao, output), 4), '| Recal:', round(recall_score(validacao, output), 4), '| mse:', round(mean_squared_error(validacao, output), 4), '| MLP:', ll, '| Tempo:', datetime.now()-dt1, '| ds:', nome_output, '| Iters:', mlp.n_iter_)

In [ ]:
#depois de validado o modelo, chama ele para classificação de outras notas
joblib.dump(last_mlp, nome_output + '.pkl', compress=7) # exporta o modelo treinado

Ideia 2

In [ ]:
#demonstração da lógica fuzz 
fuzz.ratio("so palo", "São Paulo")

62

Ideia 3

Fazer reconhecimento apenas por imagem do brasão da cidade